In [1]:
# draw_qrc_circuit.py
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.visualization import circuit_drawer
import os

# --- QRC パラメータ（ノートブックの値と同一） ---
RZ_BASE = 0.07   # RZの係数: (q+1)*0.07
RX_BASE = 0.13   # RXの係数: (q+1)*0.13

def append_timeslice(qc: QuantumCircuit, thetas, depth: int = 2, kappa: float = 1.0):
    """
    1タイムスライス分の QRC を qc に追記する。
    先頭: 各量子ビットへ RY(kappa * theta[q]) を注入
    続く depth 回: RZ → 隣接CX鎖＋リング結合 → RX
    """
    n = qc.num_qubits
    # --- 1) Ry埋め込み ---
    for q in range(n):
        qc.ry(kappa * thetas[q], q)

    # --- 2) ミキシング層（depth 回繰り返し）---
    for _ in range(depth):
        # (a) Z回転
        for q in range(n):
            qc.rz((q + 1) * RZ_BASE, q)
        # (b) 隣接 CNOT 鎖
        for q in range(n - 1):
            qc.cx(q, q + 1)
        # (c) 末端→先頭のリング結合
        qc.cx(n - 1, 0)
        # (d) X回転
        for q in range(n):
            qc.rx((q + 1) * RX_BASE, q)

def build_qrc_timeslice(n_qubits: int = 6, depth: int = 2, kappa: float = 1.0):
    """1タイムスライスだけ展開した詳細図（スライドで構造説明用）。"""
    qc = QuantumCircuit(n_qubits, name="QRC_TimeSlice")
    thetas = ParameterVector("theta_s0", n_qubits)  # s=0 のパラメータ
    append_timeslice(qc, thetas, depth=depth, kappa=kappa)
    return qc

def build_qrc_unrolled(W: int = 3, n_qubits: int = 6, depth: int = 2, kappa: float = 1.0, add_barrier: bool = True):
    """
    タイムスライスを W 回「そのまま」展開した図。
    W を大きくすると横に長くなるので、スライド用には 2〜3 を推奨。
    """
    qc = QuantumCircuit(n_qubits, name=f"QRC_W{W}")
    for s in range(W):
        thetas = ParameterVector(f"theta_s{s}", n_qubits)
        append_timeslice(qc, thetas, depth=depth, kappa=kappa)
        if add_barrier and s < W - 1:
            qc.barrier()
    return qc

def export_circuit(qc: QuantumCircuit, out_dir: str, basename: str):
    os.makedirs(out_dir, exist_ok=True)
    # 折り返しなしで1行表示（fold=-1）。幅が長すぎる場合は fold に整数を入れて改行させる。
    fig = circuit_drawer(qc, output="mpl", fold=-1, style="iqx")
    fig.savefig(os.path.join(out_dir, f"{basename}.png"), dpi=300, bbox_inches="tight")
    fig.savefig(os.path.join(out_dir, f"{basename}.svg"), bbox_inches="tight")
    print(f"[SAVE] {os.path.join(out_dir, f'{basename}.png')}")
    print(f"[SAVE] {os.path.join(out_dir, f'{basename}.svg')}")

if __name__ == "__main__":
    OUT = "qrc_figs"

    # 1) 構造説明用（1タイムスライスのみ詳細）
    qc_slice = build_qrc_timeslice(n_qubits=6, depth=2, kappa=1.0)
    export_circuit(qc_slice, OUT, "qrc_timeslice_detail")

    # 2) 実行系列イメージ（W=3 で展開；長くなるので小さめ推奨）
    qc_unrolled = build_qrc_unrolled(W=3, n_qubits=6, depth=2, kappa=1.0, add_barrier=True)
    export_circuit(qc_unrolled, OUT, "qrc_unrolled_W3")


C:\qiskit-python\venv312\Lib\site-packages\qiskit\visualization\circuit\matplotlib.py:272: UserWarning: Style JSON file 'iqx.json' not found in any of these locations: C:\qiskit-python\venv312\Lib\site-packages\qiskit\visualization\circuit\styles\iqx.json, iqx.json. Will use default style.
  self._style, def_font_ratio = load_style(


[SAVE] qrc_figs\qrc_timeslice_detail.png
[SAVE] qrc_figs\qrc_timeslice_detail.svg
[SAVE] qrc_figs\qrc_unrolled_W3.png
[SAVE] qrc_figs\qrc_unrolled_W3.svg


In [3]:
# vqa_circuit_only.py
import math
import os
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector

def build_vqa_circuit(
    n_qubits: int = 6,
    depth: int = 2,
    reupload: int = 2,
    add_zz: bool = True,
    zz_angle: float = 0.2,
    x_scale: float = math.pi/6,
):
    """
    H^⊗n → [ reupload 回:
       RY(x_scale * x_i)  # angle encoding
       → depth 回: { RX(θ)→RY(θ) 全qubit → 隣接CX鎖 → (任意)RZZ(zz_angle) }
    ]
    """
    qc = QuantumCircuit(n_qubits, name="VQA circuit")

    # 1) H で初期化
    for q in range(n_qubits):
        qc.h(q)

    # 2) パラメータ（x: 入力埋め込み, theta: 学習パラメータ）
    x = ParameterVector("x", n_qubits)
    theta = ParameterVector("theta", 2 * n_qubits * depth * max(1, int(reupload)))
    tidx = 0

    # 3) 再アップロード + 各層
    for _ in range(reupload):
        # 角度エンコード RY(π/6 * x_i)
        for q in range(n_qubits):
            qc.ry(x_scale * x[q], q)
        qc.barrier()

        # depth 層
        for _ in range(depth):
            # 単一量子ビット回転
            for q in range(n_qubits):
                qc.rx(theta[tidx], q); tidx += 1
                qc.ry(theta[tidx], q); tidx += 1

            # 隣接 CX チェーン
            for q in range(n_qubits - 1):
                qc.cx(q, q + 1)

            # 任意で ZZ 結合
            if add_zz:
                for q in range(n_qubits - 1):
                    qc.rzz(zz_angle, q, q + 1)

            qc.barrier()

    return qc, x, theta

def export_slide_figures(qc, basename="vqa_circuit", outdir="VQAfig",
                         fold=120, dpi=400):
    """
    fold は回路の折返し幅（広いほど 1 行に収まりやすい）
    出力は ./VQAfig に png/svg を保存
    """
    os.makedirs(outdir, exist_ok=True)
    png_path = os.path.join(outdir, f"{basename}.png")
    svg_path = os.path.join(outdir, f"{basename}.svg")
    fig = qc.draw("mpl", fold=fold, idle_wires=False)
    fig.savefig(png_path, dpi=dpi, bbox_inches="tight")
    fig.savefig(svg_path, bbox_inches="tight")
    print(f"Saved: {png_path}\nSaved: {svg_path}")

if __name__ == "__main__":
    # ノートブックと同じ構成で回路を生成
    qc, x, theta = build_vqa_circuit()

    # ① パラメータ記号のまま（式を見せたいスライド向け）
    export_slide_figures(qc, basename="vqa_circuit_symbolic", outdir="VQAfig")

    # ② 数値を埋めてラベルを簡素に（角度=0 の例。任意に変更可）
    bind = {**{x[i]: 0.0 for i in range(len(x))},
            **{theta[j]: 0.0 for j in range(len(theta))}}
    # バージョン差異に対応
    try:
        qc_bound = qc.bind_parameters(bind)
    except AttributeError:
        qc_bound = qc.assign_parameters(bind, inplace=False)

    export_slide_figures(qc_bound, basename="vqa_circuit_numeric", outdir="VQAfig")


Saved: VQAfig\vqa_circuit_symbolic.png
Saved: VQAfig\vqa_circuit_symbolic.svg
Saved: VQAfig\vqa_circuit_numeric.png
Saved: VQAfig\vqa_circuit_numeric.svg
